In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('persian_digits.h5')

# Read the input image (make sure it's a binary image)
image = cv2.imread('images/persian-digits.jpg', cv2.IMREAD_GRAYSCALE)
image_annotated = cv2.imread('images/persian-digits.jpg')

# Apply thresholding to create a binary image
_, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
binary_image = 255-binary_image

# Use connectedComponentsWithStats to obtain labels and bounding boxes
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

# Iterate through each connected component
for label in range(1, num_labels):
    # Get the bounding box of the current connected component
    x, y, w, h = stats[label][:4]

    if h>5 and w>5:
        # Crop the connected component using the bounding box
        cropped_img = binary_image[y:y+h, x:x+w]
        resized_img = cv2.resize(cropped_img, (10,10))
        reshaped_img = resized_img.reshape(1, 100)
        x_ = reshaped_img.astype('float32')/255 #normalization
        p = model.predict(x_)
        result= np.argmax(p)
        print(result)
        y = y-10 if y>10 else y+10
        cv2.putText(image_annotated, str(result), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

# Display or save the cropped connected component
cv2.imshow('result image', image_annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Asus\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


1/1 [==============================] - 0s 448ms/step
2
1/1 [==============================] - 0s 46ms/step
3
1/1 [==============================] - 0s 40ms/step
6
1/1 [==============================] - 0s 44ms/step
7


In [3]:
result

7

In [4]:
np.argmax(result)

0

In [5]:
stats

array([[     0,      0,    700,    525, 365699],
       [   181,    215,     38,     71,    447],
       [   386,    227,     48,     55,    475],
       [   531,    307,     28,     57,    435],
       [   379,    308,      1,      2,      2],
       [   262,    349,     47,     43,    439],
       [   344,    377,      1,      1,      1],
       [   343,    380,      2,      1,      2]], dtype=int32)

In [6]:
cap = cv2.VideoCapture(0)
model = keras.models.load_model('persian_digits.h5')

while True:
    ret, color_frame = cap.read()
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    cv2.GaussianBlur(gray_frame, (7,7), 0)
    # Apply thresholding to create a binary image
    _, binary_image = cv2.threshold(gray_frame, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary_image = 255-binary_image
    # Use connectedComponentsWithStats to obtain labels and bounding boxes
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
        # Iterate through each connected component
    for label in range(1, num_labels):
        # Get the bounding box of the current connected component
        x, y, w, h = stats[label][:4]
    
        if h>5 and w>5:
            # Crop the connected component using the bounding box
            cropped_img = binary_image[y:y+h, x:x+w]
            resized_img = cv2.resize(cropped_img, (10,10))
            _, bimage = cv2.threshold(resized_img, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            #dilation = cv2.dilate(bimage, np.ones((5,5),np.uint8), iterations =1)
            closing = cv2.morphologyEx(bimage, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
            #reshaped_img = dilation.reshape(1, 100)
            reshaped_img = closing.reshape(1, 100)
            x_ = reshaped_img.astype('float32')/255 #normalization
            p = model.predict(x_)
            if np.max(p)<0.6:
                continue
            result= np.argmax(p)
            print(result)
            y = y-10 if y>10 else y+10
            cv2.putText(color_frame, str(result), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

    cv2.imshow('Webcam', color_frame)
    cv2.imshow('Webcam binary', binary_image)
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
# Release camera and close windows
cap.release()
cv2.destroyAllWindows()  

1/1 [==============================] - 0s 100ms/step
1
1/1 [==============================] - 0s 42ms/step
1
1/1 [==============================] - 0s 39ms/step
5
1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 56ms/step
1
1/1 [==============================] - 0s 57ms/step
1
1/1 [==============================] - 0s 50ms/step
1
1/1 [==============================] - 0s 56ms/step
9
1/1 [==============================] - 0s 52ms/step
1
1/1 [==============================] - 0s 54ms/step
1
1/1 [==============================] - 0s 53ms/step
1
1/1 [==============================] - 0s 53ms/step
6
1/1 [==============================] - 0s 43ms/step
1
1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 37ms/step
1
1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 40ms/step
6
1/1 [==============================] - 0s 45ms/step
1
1/1 [======================

KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()

In [2]:
cap = cv2.VideoCapture(0)
model = keras.models.load_model('MNIST_digits.h5')

while True:
    ret, color_frame = cap.read()
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    cv2.GaussianBlur(gray_frame, (7,7), 0)
    # Apply thresholding to create a binary image
    _, binary_image = cv2.threshold(gray_frame, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary_image = 255-binary_image
    # Use connectedComponentsWithStats to obtain labels and bounding boxes
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
        # Iterate through each connected component
    for label in range(1, num_labels):
        # Get the bounding box of the current connected component
        x, y, w, h = stats[label][:4]
    
        if h>5 and w>5:
            # Crop the connected component using the bounding box
            cropped_img = binary_image[y:y+h, x:x+w]
            resized_img = cv2.resize(cropped_img, (10,10))
            _, bimage = cv2.threshold(resized_img, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            #dilation = cv2.dilate(bimage, np.ones((5,5),np.uint8), iterations =1)
            closing = cv2.morphologyEx(bimage, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
            #reshaped_img = dilation.reshape(1, 100)
            reshaped_img = closing.reshape(1, 100)
            x_ = reshaped_img.astype('float32')/255 #normalization
            p = model.predict(x_)
            if np.max(p)<0.6:
                continue
            result= np.argmax(p)
            print(result)
            y = y-10 if y>10 else y+10
            cv2.putText(color_frame, str(result), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

    cv2.imshow('Webcam', color_frame)
    cv2.imshow('Webcam binary', binary_image)
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
# Release camera and close windows
cap.release()
cv2.destroyAllWindows()  

ValueError: in user code:

    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 o

In [7]:
import cv2
import numpy as np
from tensorflow import keras

# Load MNIST model
model = keras.models.load_model('MNIST_digits.h5')  # اطمینان حاصل کنید مدل برای MNIST آموزش دیده است

cap = cv2.VideoCapture(0)

while True:
    ret, color_frame = cap.read()
    if not ret:
        break
        
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_frame, (7,7), 0)
    
    # Apply adaptive thresholding
    binary_image = cv2.adaptiveThreshold(blurred, 255, 
                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY_INV, 11, 2)
    
    # Find contours
    contours, _ = cv2.findContours(binary_image.copy(), 
                                 cv2.RETR_EXTERNAL, 
                                 cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # Filter small contours
        if w >= 20 and h >= 20:
            # Extract and preprocess the digit
            digit = binary_image[y:y+h, x:x+w]
            
            # Resize to match MNIST input size (28x28)
            resized_digit = cv2.resize(digit, (28, 28))
            
            # Flatten the image to (784,) and normalize
            flattened_digit = resized_digit.reshape(1, 784).astype('float32') / 255.0
            
            # Predict
            try:
                pred = model.predict(flattened_digit)
                predicted_digit = np.argmax(pred)
                confidence = np.max(pred)
                
                # Only display if confidence is high enough
                if confidence > 0.8:
                    # Draw rectangle and label
                    cv2.rectangle(color_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(color_frame, str(predicted_digit), 
                               (x, y-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 
                               1, (0, 255, 0), 2)
            except Exception as e:
                print(f"Prediction error: {e}")
    
    # Display
    cv2.imshow('MNIST Digit Recognition', color_frame)
    cv2.imshow('Processed View', binary_image)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step


KeyboardInterrupt: 

In [8]:
cap.release()
cv2.destroyAllWindows()